In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q


In [10]:
from datasets import load_dataset

multi_news_dataset = load_dataset("alexfabbri/multi_news")
multi_news_dataset

/home/pavithra/.local/lib/python3.8/site-packages/datasets/load.py:1486: FutureWarning: The repository for alexfabbri/multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/alexfabbri/multi_news
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [26]:
len(multi_news_dataset["train"]["document"][100].split())

1005

In [5]:
# gonna use this model
# Load model directly
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail", from_pt=True)


2024-05-26 18:10:46.533706: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 393637888 exceeds 10% of free system memory.
2024-05-26 18:10:47.033588: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 393637888 exceeds 10% of free system memory.
2024-05-26 18:10:47.102048: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 393637888 exceeds 10% of free system memory.
2024-05-26 18:10:53.401002: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 393637888 exceeds 10% of free system memory.
2024-05-26 18:10:54.554373: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 393637888 exceeds 10% of free system memory.
All PyTorch model weights were used when initializing TFPegasusForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFPegasusForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed

In [7]:
# play with the tockenizer a little bit
a = tokenizer("somehting for my understanding")
print(a)
tokenizer.convert_ids_to_tokens(a["input_ids"])



{'input_ids': [181, 1467, 4011, 118, 161, 1301, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


['▁some', 'h', 'ting', '▁for', '▁my', '▁understanding', '</s>']

In [11]:
type(multi_news_dataset)

datasets.dataset_dict.DatasetDict

: 

In [ ]:
from transformers.tokenization_utils import BatchEncoding
from datasets.dataset_dict import DatasetDict

def tokenize_sentence(dataset: DatasetDict, input_feature: str, response_feature: str) -> BatchEncoding:
    model_input = tokenizer(dataset[input_feature])
    print(type(model_input))
    labels = tokenizer(dataset[response_feature])
    print(labels)
    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_data = multi_news_dataset.map(tokenize_sentence, batched=True)
print(type(tokenized_data))